In [2]:
%pylab inline
from classy import *

Populating the interactive namespace from numpy and matplotlib
Version:  0.0.25


In [29]:
from NumPyNet.network import Network
from NumPyNet.layers.connected_layer import Connected_layer
from NumPyNet.layers.cost_layer import Cost_layer
from NumPyNet.optimizer import Adam,SGD
from NumPyNet.utils import to_categorical
from NumPyNet.utils import from_categorical
from NumPyNet.metrics import mean_accuracy_score

In [36]:
data=load_excel('data/iris.xls')
data_train,data_test=split(data,test_size=0.2)

iris.data 151 5
150 vectors of length 4
Feature names: 'petal length in cm', 'petal width in cm', 'sepal length in cm', 'sepal width in cm'
Target values given.
Target names: 'Iris-setosa', 'Iris-versicolor', 'Iris-virginica'
Mean:  [3.75866667 1.19866667 5.84333333 3.054     ]
Median:  [4.35 1.3  5.8  3.  ]
Stddev:  [1.75852918 0.76061262 0.82530129 0.43214658]
Original vector shape:  (150, 4)
Train vector shape:  (120, 4)
Test vector shape:  (30, 4)


In [37]:
X=data.vectors
Y=data.targets

In [38]:
X.shape

(150, 4)

In [39]:
Y.shape

(150,)

In [40]:
Y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [41]:
one_hot_y=to_categorical(Y)
one_hot_y[40:60,:]

array([[1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.]])

In [42]:
def accuracy (y_true, y_pred):
    '''
    Temporary metrics to overcome "from_categorical" missing in standard metrics
    '''
    from NumPyNet.metrics import mean_accuracy_score
    truth = from_categorical(y_true)
    predicted = from_categorical(y_pred)
    return mean_accuracy_score(truth, predicted)


In [43]:
X_train=data_train.vectors
y_train=data_train.targets
one_hot_y_train=to_categorical(y_train)

X_test=data_test.vectors
y_test=data_test.targets
one_hot_y_test=to_categorical(y_test)

In [44]:
# Reshape the data according to a 4D tensor
num_samples, size = X_test.shape
num_samples, size

num_classes=one_hot_y_test.shape[1]

X_test = X_test.reshape(num_samples, 1, 1, size)
one_hot_y_test = one_hot_y_test.reshape(num_samples,1,1,-1)

X_test.shape,one_hot_y_test.shape

((30, 1, 1, 4), (30, 1, 1, 3))

In [45]:
# Reshape the data according to a 4D tensor
num_samples, size = X_train.shape
num_samples, size

num_classes=one_hot_y_train.shape[1]

X_train = X_train.reshape(num_samples, 1, 1, size)
one_hot_y_train = one_hot_y_train.reshape(num_samples,1,1,-1)

X_train.shape,one_hot_y_train.shape

((120, 1, 1, 4), (120, 1, 1, 3))

In [46]:
batch=len(y_train)
step = batch

In [54]:
model = Network(batch=batch, input_shape=X_train.shape[1:])
model.add(Connected_layer(outputs=4, activation='relu'))
model.add(Connected_layer(outputs=num_classes, activation='linear'))
model.add(Cost_layer(cost_type='mse'))

#model.compile(optimizer=SGD(lr=0.1, decay=0.03, lr_min=0., lr_max=np.inf))
model.compile(optimizer=Adam(), metrics=[accuracy])

In [55]:
model.summary()

layer       filters  size              input                output
   0 input                   120 x   1 x   1 x   4   ->   120 x   1 x   1 x   4
   1 connected               120 x   1 x   1 x   4   ->   120 x   4
   2 connected               120 x   1 x   1 x   4   ->   120 x   3
   3 cost                    120 x   1 x   1 x   3   ->   120 x   1 x   1 x   3


In [56]:
model.fit(X=X_train, y=one_hot_y_train, max_iter=10000,verbose=False)

In [57]:
model.batch

120

In [58]:
model.batch=len(X_test)
loss, out = model.evaluate(X=X_test, truth=one_hot_y_test, verbose=True)

 30/30  |██████████████████████████████████████████████████| (0.0 sec/iter) loss: 0.001
Prediction on 30 samples took 0.0 sec


In [59]:
truth = from_categorical(one_hot_y_test)
predicted = from_categorical(out)
performance=mean_accuracy_score(truth, predicted)

print('\nLoss Score: {:.3f}'.format(loss))
print('Accuracy Score: {:.3f}'.format(performance))


Loss Score: 0.001
Accuracy Score: 0.900


In [60]:
model = Network(batch=batch, input_shape=X_train.shape[1:])
model.add(Connected_layer(outputs=num_classes, activation='logistic'))
model.add(Cost_layer(cost_type='mse'))

#model.compile(optimizer=SGD(lr=0.1, decay=0.03, lr_min=0., lr_max=np.inf))
model.compile(optimizer=Adam(), metrics=[accuracy])
model.summary()

layer       filters  size              input                output
   0 input                   120 x   1 x   1 x   4   ->   120 x   1 x   1 x   4
   1 connected               120 x   1 x   1 x   4   ->   120 x   3
   2 cost                    120 x   1 x   1 x   3   ->   120 x   1 x   1 x   3


In [61]:
model.fit(X=X_train, y=one_hot_y_train, max_iter=10000,verbose=False)

In [62]:
truth = from_categorical(one_hot_y_test)
predicted = from_categorical(out)
performance=mean_accuracy_score(truth, predicted)

print('\nLoss Score: {:.3f}'.format(loss))
print('Accuracy Score: {:.3f}'.format(performance))


Loss Score: 0.001
Accuracy Score: 0.900


## Images

In [63]:
images=image.load_images('data/digits')

[9]: 180 files found
[0]: 178 files found
[7]: 179 files found
[6]: 181 files found
[1]: 182 files found
[8]: 174 files found
[4]: 181 files found
[3]: 183 files found
[2]: 177 files found
[5]: 182 files found


In [81]:
data=image.images_to_vectors(images)

1797 vectors of length 64
Feature names: 'p0', 'p1', 'p2', 'p3', 'p4'  , ... ,  'p59', 'p60', 'p61', 'p62', 'p63'  (64 features)
Target values given.
Target names: '9', '0', '7', '6', '1', '8', '4', '3', '2', '5'
Mean:  [0.00000000e+00 4.71341124e+00 8.25158598e+01 1.88401224e+02
 1.88589872e+02 9.17840846e+01 2.15581525e+01 2.04785754e+00
 8.51419032e-02 3.14913745e+01 1.65203116e+02 1.90677240e+02
 1.63504174e+02 1.30023372e+02 2.92036728e+01 1.70005565e+00
 4.22927101e-02 4.11185309e+01 1.57575960e+02 1.11080690e+02
 1.12814691e+02 1.24173066e+02 2.82609905e+01 7.82971619e-01
 1.66944908e-02 3.90011130e+01 1.44642181e+02 1.40286589e+02
 1.58007791e+02 1.20022816e+02 3.66889260e+01 3.33889816e-02
 0.00000000e+00 3.70083472e+01 1.21928770e+02 1.44347245e+02
 1.63978854e+02 1.39081803e+02 4.60222593e+01 0.00000000e+00
 1.37451308e-01 2.49860879e+01 1.09460768e+02 1.14961046e+02
 1.21993879e+02 1.30974958e+02 5.47762938e+01 4.24040067e-01
 1.13522538e-01 1.10378408e+01 1.19312187e+02 1.

In [83]:
data.vectors/=255
summary(data)

1797 vectors of length 64
Feature names: 'p0', 'p1', 'p2', 'p3', 'p4'  , ... ,  'p59', 'p60', 'p61', 'p62', 'p63'  (64 features)
Target values given.
Target names: '9', '0', '7', '6', '1', '8', '4', '3', '2', '5'
Mean:  [0.00000000e+00 7.24861398e-05 1.26898669e-03 2.89736600e-03
 2.90026716e-03 1.41151995e-03 3.31536370e-04 3.14933878e-05
 1.30937183e-06 4.84296417e-04 2.54060925e-03 2.93236816e-03
 2.51448172e-03 1.99959050e-03 4.49114537e-04 2.61446466e-05
 6.50406922e-07 6.32349571e-04 2.42331349e-03 1.70827666e-03
 1.73494335e-03 1.90962040e-03 4.34617309e-04 1.20410860e-05
 2.56739574e-07 5.99786435e-04 2.22440879e-03 2.15742543e-03
 2.42995449e-03 1.84579494e-03 5.64228004e-04 5.13479149e-07
 0.00000000e+00 5.69140288e-04 1.87510604e-03 2.21987306e-03
 2.52178168e-03 2.13889739e-03 7.07762542e-04 0.00000000e+00
 2.11382249e-06 3.84253563e-04 1.68336437e-03 1.76795150e-03
 1.87610732e-03 2.01422466e-03 8.42388217e-04 6.52118519e-06
 1.74582911e-06 1.69747649e-04 1.83486639e-03 2.

In [84]:
data_train,data_test=split(data)

Original vector shape:  (1797, 64)
Train vector shape:  (1347, 64)
Test vector shape:  (450, 64)


In [85]:
X_train=data_train.vectors
y_train=data_train.targets
one_hot_y_train=to_categorical(y_train)

X_test=data_test.vectors
y_test=data_test.targets
one_hot_y_test=to_categorical(y_test)

In [86]:
one_hot_y_test[:10,:]

array([[0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.]])

In [87]:
# Reshape the data according to a 4D tensor
num_samples, size = X_test.shape
num_samples, size

num_classes=one_hot_y_test.shape[1]

X_test = X_test.reshape(num_samples, 1, 1, size)
one_hot_y_test = one_hot_y_test.reshape(num_samples,1,1,-1)

X_test.shape,one_hot_y_test.shape

((450, 1, 1, 64), (450, 1, 1, 10))

In [88]:
# Reshape the data according to a 4D tensor
num_samples, size = X_train.shape
num_samples, size

num_classes=one_hot_y_train.shape[1]

X_train = X_train.reshape(num_samples, 1, 1, size)
one_hot_y_train = one_hot_y_train.reshape(num_samples,1,1,-1)

X_train.shape,one_hot_y_train.shape

((1347, 1, 1, 64), (1347, 1, 1, 10))

In [89]:
batch=len(y_train)
step = batch

In [90]:
model = Network(batch=batch, input_shape=X_train.shape[1:])
model.add(Connected_layer(outputs=num_classes, activation='logistic'))
model.add(Cost_layer(cost_type='mse'))

#model.compile(optimizer=SGD(lr=0.1, decay=0.03, lr_min=0., lr_max=np.inf))
model.compile(optimizer=Adam(), metrics=[accuracy])
model.summary()

layer       filters  size              input                output
   0 input                  1347 x   1 x   1 x  64   ->  1347 x   1 x   1 x  64
   1 connected              1347 x   1 x   1 x  64   ->  1347 x  10
   2 cost                   1347 x   1 x   1 x  10   ->  1347 x   1 x   1 x  10


In [91]:
model.fit(X=X_train, y=one_hot_y_train, max_iter=10000,verbose=False)

In [92]:
model.batch=len(X_test)
loss, out = model.evaluate(X=X_test, truth=one_hot_y_test, verbose=True)

450/450 |██████████████████████████████████████████████████| (0.0 sec/iter) loss: 0.000
Prediction on 450 samples took 0.0 sec


In [93]:
truth = from_categorical(one_hot_y_test)
predicted = from_categorical(out)
performance=mean_accuracy_score(truth, predicted)

print('\nLoss Score: {:.3f}'.format(loss))
print('Accuracy Score: {:.3f}'.format(performance))


Loss Score: 0.000
Accuracy Score: 0.582


## convo net

In [95]:
from NumPyNet.layers.convolutional_layer import Convolutional_layer
from NumPyNet.layers.maxpool_layer import Maxpool_layer
from NumPyNet.layers.softmax_layer import Softmax_layer
from NumPyNet.layers.batchnorm_layer import BatchNorm_layer


In [97]:
batch=128

model = Network(batch=batch, input_shape=X_train.shape[1:])
model.add(Convolutional_layer(size=3, filters=32, stride=1, pad=True, activation='Relu'))
model.add(BatchNorm_layer())
model.add(Maxpool_layer(size=2, stride=1, padding=True))
model.add(Connected_layer(outputs=100, activation='Relu'))
model.add(BatchNorm_layer())
model.add(Connected_layer(outputs=num_classes, activation='Linear'))
model.add(Softmax_layer(spatial=True, groups=1, temperature=1.))
model.add(Cost_layer(cost_type=cost_type.mse))

# model.compile(optimizer=SGD(lr=0.01, decay=0., lr_min=0., lr_max=np.inf))
model.compile(optimizer=Adam(), metrics=[accuracy])
model.summary()

/Users/bblais/opt/anaconda3/envs/py3/lib/python3.7/site-packages/NumPyNet/layers/connected_layer.py:73: RuntimeWarning: divide by zero encountered in true_divide
  scale = np.sqrt(2. / self.inputs)


OverflowError: Range exceeds valid bounds

In [98]:
X_train.shape[1:]

(1, 1, 64)